In [157]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from pandas_profiling import ProfileReport
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
import datetime 
from dateutil.parser import parse
import nbformat
from sklearn import linear_model

In [158]:
#mass index modifier
def set_indexes(dfs, index_list):
    for df, index in zip(dfs, index_list):
        df.set_index(index, inplace=True, drop=True)

In [163]:
RAW_DATA_FOLDER = 'data/raw/'
OUTPUT_DATA_FOLDER = 'data/output/'
REPORT_FOLDER = 'reports/'

"""test line"""

#sectors and countries that we are interested
sector_list = ['Energy Storage & Management', 'Industry 4.0', 'Mobility & Automotive', 'Supply Chain & Logistics', 'Other']
country_list = ['Germany', 'Turkey', 'United Kingdom', 'Other']

pipeline = pd.read_csv(os.path.join(RAW_DATA_FOLDER, 'pipeline.csv'))

In [164]:
#standardize the date input
pipeline.rename(columns={'Day Created': 'Date'}, inplace=True)
pipeline['Date'] = pipeline['Date'].str.replace('.', '-')
pipeline['Date'] = pipeline['Date'].str.replace('/', '-')
pipeline.loc[~pipeline['Country'].isin(country_list), 'Country'] = 'Other'
#get rid of irrelevant rows
pipeline.dropna(subset = ['Sector'], inplace=True)
#adjust time format
pipeline["Date"] = pd.to_datetime(pipeline["Date"]).dt.strftime('%Y-%m-%d')
pipeline["Date"] = pipeline["Date"].apply(lambda x : parse(x))

In [180]:
def forecast_for_item(item, data_path):
    country_cumulative = pd.read_csv(data_path)
    country_cumulative = country_cumulative[['Date', item]]

    country_cumulative['Date'] = pd.to_datetime(country_cumulative['Date'], format="%Y-%m-%d")

    minimum_date = country_cumulative['Date'].min()
    maximum_date = country_cumulative['Date'].max()
    country_cumulative['Date'] = country_cumulative['Date'] - minimum_date

    country_cumulative['Date'] = country_cumulative['Date'].apply(lambda x : int(str(x).replace(" days 00:00:00", "")))
    country_cumulative[item] = country_cumulative[item].apply(lambda x : int(x))

    X = country_cumulative[['Date']]
    y = country_cumulative[item]

    model = linear_model.LinearRegression().fit(X, y)

    days_later_from_earliest = [[365*2], [365*3], [365*5], [365*10]]

    return model.predict(np.array(days_later_from_earliest))   

In [186]:
#earliest: "2018-01-04"
#latest: "2020-12-22"
predictions = forecast_for_item('Other', os.path.join(OUTPUT_DATA_FOLDER, 'cumulative_data_Country.csv'))
print(predictions)

[ 126.93260815  236.9754363   457.06109261 1007.27523338]
